In [34]:
import os
import time
import copy
import datetime
import numpy as np
import pandas as pd
from numpy.random import seed
from utils2 import split_data
from utils2.tokeniser import Tokenizer
from utils2.utils import get_class_weights
from utils2.embed import load_glove_tw_vectors
from utils2.users import load_users, select_post, combine_users
from sklearn.model_selection import StratifiedKFold
from nltk.tokenize import TweetTokenizer
from keras.optimizers import Adam#, AdaMod
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint
#from keras.preprocessing.sequence import pad_sequences
#from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Conv1D, GlobalMaxPooling1D
from keras.layers import Embedding, Dropout, Input
from keras.models import Model
#from tensorflow import set_random_seed
import tensorflow as tf
from keras import backend as K


def create_dl_model():
    num_filters = 100
    drop = 0.5

    embedding_layer = Embedding(num_words,
                                EMBEDDING_DIM,
                                input_length=INPUT_LENGTH,
                                weights=[embedding_matrix],
                                trainable=False)
                                # trainable=True)

    model_input = Input(shape=(INPUT_LENGTH,), dtype='int32')
    model_layer = embedding_layer(model_input)
    model_layer = Dropout(drop)(model_layer)
    model_layer = Conv1D(filters=num_filters, kernel_size=3)(model_layer)
    model_layer = GlobalMaxPooling1D()(model_layer)
    model_layer = Dropout(drop)(model_layer)
    model_layer = Dense(100, activation='relu')(model_layer)
    model_layer = Dropout(drop)(model_layer)
    model_layer = Dense(2, activation='softmax')(model_layer)
    model = Model(model_input, model_layer)

    print(model.summary())
    adam = Adam(lr=0.007, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    # adam = AdaMod(lr=0.007)
    model.compile(loss='binary_crossentropy',
                  optimizer=adam,
                  metrics=['acc'])  # metrics.binary_accuracy
    return model


SEED = 1
seed(SEED)
tf.random.set_seed(SEED)
#set_random_seed(SEED)

POST_SIZE = 100
MAX_POSTS = 3200
MAX_NB_WORDS = 50000
INPUT_LENGTH = 35000
EMBEDDING_DIM = 100
NB_DEPRESS = 1984

DIR = '../data/'

inputs, labels = load_users(DIR, POST_SIZE, MAX_POSTS, NB_DEPRESS, True)
control = inputs[inputs.label == 0].reset_index(drop=True).copy()
depress = inputs[inputs.label == 1].reset_index(drop=True).copy()
del inputs, labels

embeddings_index = load_glove_tw_vectors(EMBEDDING_DIM)  # load pre-trained embedding vectors

i = 0
MODEL_FOLDER = "logs/{}/{}".format('DL', datetime.datetime.now().strftime("%Y-%m-%d %H-%M") + '-' + str(MAX_NB_WORDS) + '-' + str(INPUT_LENGTH))
for data_index in split_data.split(depress, 4):
    data_fold = depress[depress.userid.isin(data_index)].copy()

    print(data_index[:10])
    print(data_fold.head())

    combine, labels = combine_users(control, data_fold)
    inputs, labels = select_post(combine, labels, MAX_POSTS)
#     del combine, data_fold
    break

FileNotFoundError: [Errno 2] No such file or directory: '../data/final_replace_depress0220.csv'

In [ ]:
import os
import time
import copy
import datetime
import numpy as np
import pandas as pd
from numpy.random import seed
from utils import split_data
from utils.tokeniser import Tokenizer
from utils.utils import get_class_weights
from utils.embed import load_glove_tw_vectors
from utils.users import load_users, select_post, combine_users
from sklearn.model_selection import StratifiedKFold
from nltk.tokenize import TweetTokenizer
from keras.optimizers import Adam, AdaMod
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Conv1D, GlobalMaxPooling1D
from keras.layers import Embedding, Dropout, Input
from keras.models import Model
from tensorflow import set_random_seed
import tensorflow as tf
from keras import backend as K


def create_dl_model():
    num_filters = 250
    drop = 0.5

    embedding_layer = Embedding(num_words,
                                EMBEDDING_DIM,
                                input_length=INPUT_LENGTH,
                                weights=[embedding_matrix],
                                trainable=False)
                                # trainable=True)

    model_input = Input(shape=(INPUT_LENGTH,), dtype='int32')
    model_layer = embedding_layer(model_input)
    model_layer = Dropout(drop)(model_layer)
    model_layer = Conv1D(filters=num_filters, kernel_size=3, activation='relu')(model_layer)
    model_layer = GlobalMaxPooling1D()(model_layer)
    model_layer = Dense(num_filters, activation='relu')(model_layer)
    model_layer = Dropout(drop)(model_layer)
    model_layer = Dense(2, activation='softmax')(model_layer)
    # model_layer = Dense(1, activation='sigmoid')(model_layer)
    model = Model(model_input, model_layer)

    print(model.summary())
#     adam = Adam(lr=0.007, clipvalue=7)
    adam = AdaMod()
    model.compile(loss='binary_crossentropy',
                  optimizer=adam,
                  metrics=['acc'])  # metrics.binary_accuracy
    return model


SEED = 1
seed(SEED)
set_random_seed(SEED)

POST_SIZE = 100
MAX_POSTS = 3200
MAX_NB_WORDS = 50000
INPUT_LENGTH = 35000
EMBEDDING_DIM = 100
NB_DEPRESS = 1984

DIR = '../data/'

inputs, labels = load_users(DIR, POST_SIZE, MAX_POSTS, NB_DEPRESS, True)
control = inputs[inputs.label == 0].reset_index(drop=True).copy()
depress = inputs[inputs.label == 1].reset_index(drop=True).copy()
del inputs, labels

embeddings_index = load_glove_tw_vectors(EMBEDDING_DIM)  # load pre-trained embedding vectors

i = 0
MODEL_FOLDER = "logs/{}/{}".format('DL', datetime.datetime.now().strftime("%Y-%m-%d %H-%M") + '-' + str(MAX_NB_WORDS) + '-' + str(INPUT_LENGTH))
for data_index in split_data.split(depress, 4):
    data_fold = depress[depress.userid.isin(data_index)].copy()

    print(data_index[:10])
    print(data_fold.head())

    combine, labels = combine_users(control, data_fold)
    inputs, labels = select_post(combine, labels, MAX_POSTS)
    del combine, data_fold

    # transform Y to categories
    print(labels.label.value_counts())
    labels = labels.label.values
    labels = to_categorical(np.asarray(labels))

    print('Input number: ', len(inputs))
    print('Label number: ', len(labels))

    # alltexts = np.hstack(np.array(inputs))
    alltexts = np.hstack(np.array(inputs).flatten())

    print('Tokenizing text')
    start = time.time()
    tknzr = TweetTokenizer(reduce_len=True)
    alltexts = [tknzr.tokenize(text.lower()) for text in alltexts]

    tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
    tokenizer.fit_on_texts(alltexts)
    word_index = tokenizer.word_index
    print('Total %s unique tokens.' % len(word_index))
    print('-----------{}-----------'.format(time.time() - start))

    print('Transforming text to input sequences')
#     start = time.time()
#     data = np.zeros((len(inputs), INPUT_LENGTH), dtype='int32')
#     for i, posts in enumerate(inputs):
#         sequences = tokenizer.texts_to_sequences([' '.join(posts)])
#         seq_data = pad_sequences(sequences, maxlen=INPUT_LENGTH, padding='pre', truncating='post')
#         data[i] = seq_data
#     del inputs, alltexts
#     print('-----------{}-----------'.format(time.time() - start))
    
    start = time.time()
    data = np.zeros((len(inputs), INPUT_LENGTH), dtype='int32')
    for i, posts in enumerate(inputs):
        ppp = []
        for j, post in enumerate(posts):
            pp = tokenizer.texts_to_sequences([post])
            if len(pp[0]) > 18:
                ppp = ppp + pp[0][:18]
            else:
                ppp = ppp + pp[0]
        sequences = [ppp]
        seq_data = pad_sequences(sequences, maxlen=INPUT_LENGTH, padding='pre', truncating='post')
        data[i] = seq_data
    del inputs, alltexts
    print('-----------{}-----------'.format(time.time() - start))

    print('Finished loading data')
    print('Shape of data tensor:', data.shape)
    print('Shape of label tensor:', labels.shape)

    num_words = min(MAX_NB_WORDS, len(word_index) + 1)

    print('Building embedding matrix')
    print('%s Selected word tokens' % num_words)

    start = time.time()
    embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
    # embedding_matrix = np.random.uniform(-1, 1, (num_words, EMBEDDING_DIM))
    for word, i in word_index.items():
        if i >= MAX_NB_WORDS:
            break
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    print('-----------{}-----------'.format(time.time() - start))

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=12345)
    for train_index, test_index in skf.split(np.asarray(data), labels[:, 1]):
        np.random.seed(0)
        tf.set_random_seed(0)
        sess = tf.Session(graph=tf.get_default_graph())
        K.set_session(sess)

        print('--------------Start training----------------{}'.format(datetime.datetime.now()))
        X_train, x_val = copy.deepcopy(data[train_index]), copy.deepcopy(data[test_index])
        y_train, y_val = copy.deepcopy(labels[train_index]), copy.deepcopy(labels[test_index])
#         del data, labels

        print('Number of positive and negative classes in training and validation set')
        print(y_train.sum(axis=0))
        print(y_val.sum(axis=0))

        class_weight = get_class_weights(np.asarray(y_train, 'int32')[:, 1])

        FOLDER = "{}/{}".format(MODEL_FOLDER, datetime.datetime.now().strftime("%Y-%m-%d %H-%M"))
        if not os.path.exists(FOLDER):
            os.makedirs(FOLDER)
        callbacks = TensorBoard(log_dir=FOLDER,
                                write_graph=True, write_grads=False, histogram_freq=0,
                                write_images=True, embeddings_freq=0, embeddings_layer_names='embedding_1',
                                embeddings_metadata=None)

        checkpoint = ModelCheckpoint(FOLDER + '/model.{epoch:02d}-{val_acc:.2f}.hdf5',
                                     verbose=0, monitor='val_acc',
                                     save_best_only=True, mode='auto')

        print("model fitting - Hierachical LSTM")

        print(class_weight)
        model = create_dl_model()

        model.fit(X_train, np.asarray(y_train, 'int32'),
        # model.fit(X_train, np.asarray(y_train[:, 1], 'int32'),
                  validation_data=(x_val, np.asarray(y_val, 'int32')),
                  # validation_data=(x_val, np.asarray(y_val[:, 1], 'int32')),
                  shuffle=False,
                  nb_epoch=200, batch_size=32, verbose=0, class_weight=class_weight, callbacks=[checkpoint, callbacks])

        model.save(FOLDER + '/my_model.h5')
        del model, X_train, y_train, x_val, y_val, class_weight
        K.clear_session()
        print('--------------Finished training----------------'.format(datetime.datetime.now()))
        break


The number of control:  1699
The number of depress:  1984
Control shape:  (3944945, 3)
Depress shape:  (4996621, 3)
Selecting users with at least 100 posts
Control users:  1699
-----------12.831624031066895-----------
Depress users:  1984
-----------17.756147623062134-----------
Labelling
Loading embedding vectors from a file
Total 1193514 word vectors.
[ 780078428687065089            25567061           416684112
           608725707  912541969968066560          2700680921
          2329059770 1064934140007538688            43903078
            16656127]
                  tweetid   userid  \
3234  1143666788107866112  5882452   
3235  1143666924246560768  5882452   
3236  1143664692679720961  5882452   
3237  1143661172891066370  5882452   
3238  1143669737378893824  5882452   

                                                  clean  label  
3234  user user i've always felt like i was kind of ...      1  
3235  user user i cannot function without my morning...      1  
3236  user way 

C:\Users\Bright\.conda\envs\py_37\lib\site-packages\ipykernel_launcher.py:200: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


--------------Finished training----------------
[         2826065615          3960721393          4318344515
          3382208403          2599342730          2192916252
           181892265 1091181448843931649  836044275712475136
            60688549]
                  tweetid   userid  \
6479  1117414019164454912  6111852   
6480  1117414158188929024  6111852   
6481  1117413476631220226  6111852   
6482  1116951765906227200  6111852   
6483  1117037694268342272  6111852   

                                                  clean  label  
6479                     good morning, twitterverse url      1  
6480  rt user: everything you need to know about k-p...      1  
6481  it's also a rhetorical trick thwimp uses all t...      1  
6482                          rt user: this thread: url      1  
6483  putin has said america is no longer a leader. ...      1  
Selecting 3200 posts from each user
-----------17.453445196151733-----------
0    1699
1     496
Name: label, dtype: int64
Input

C:\Users\Bright\.conda\envs\py_37\lib\site-packages\ipykernel_launcher.py:200: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


--------------Finished training----------------
[           22936389          1586758399          3227968831
          2731028615          3056177975            25787341
 1104780414516977666           280932608            56891919
  800940814641856512]
               tweetid  userid  \
0   589962698353299457  802310   
1  1130666193289863168  802310   
2  1070479497898520577  802310   
3  1070482628153794560  802310   
4  1070483237879799809  802310   

                                               clean  label  
0  user github is giving me more grief than the l...      1  
1  user some of us just ride the risk-train and g...      1  
2  user super happy to see another competent deve...      1  
3  user you can't trust lnp to make sensible cybe...      1  
4  user user is user prepared to stand against <h...      1  
Selecting 3200 posts from each user
-----------15.92917799949646-----------
0    1699
1     496
Name: label, dtype: int64
Input number:  2195
Label number:  2195
Tokenizi

C:\Users\Bright\.conda\envs\py_37\lib\site-packages\ipykernel_launcher.py:200: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


--------------Finished training----------------
[ 960112188433338368 1004044855931293696           497519459
            41555588  753773198383128576          2868511826
 1019322256953106437  962874724987551744 1004689374221623297
          2237542326]
                   tweetid    userid  \
22499  1089551448323575809  13379712   
22500  1089551092256571393  13379712   
22501  1089551239837294594  13379712   
22502  1089551307801800709  13379712   
22503  1089551538832449537  13379712   

                                                   clean  label  
22499  rt user: cop: your neighbor has the word butth...      1  
22500  rt user: [prison]  prisoner: what's for breakf...      1  
22501  rt user: suspect: i ain’t talkin  cop: [sharpe...      1  
22502  rt user: [lighting a cigar] man i kicked my fa...      1  
22503  rt user: me: do you wanna go skydiving  her: i...      1  
Selecting 3200 posts from each user
-----------15.560192346572876-----------
0    1699
1     496
Name: label, 

C:\Users\Bright\.conda\envs\py_37\lib\site-packages\ipykernel_launcher.py:200: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


--------------Finished training----------------


In [ ]:
def create_dl_model():
    num_filters = 250
    drop = 0.2

    embedding_layer = Embedding(num_words,
                                EMBEDDING_DIM,
                                input_length=INPUT_LENGTH,
                                weights=[embedding_matrix],
                                trainable=False)
                                # trainable=True)

    model_input = Input(shape=(INPUT_LENGTH,), dtype='int32')
    model_layer = embedding_layer(model_input)
    model_layer = Dropout(drop)(model_layer)
    model_layer = Conv1D(filters=num_filters, kernel_size=3)(model_layer)
    model_layer = GlobalMaxPooling1D()(model_layer)
#     model_layer = Dropout(drop)(model_layer)
    model_layer = Dense(num_filters, activation='relu')(model_layer)
    model_layer = Dropout(drop)(model_layer)
    model_layer = Dense(2, activation='softmax')(model_layer)
    # model_layer = Dense(1, activation='sigmoid')(model_layer)
    model = Model(model_input, model_layer)

    print(model.summary())
#     adam = Adam(lr=0.007, clipnorm=7)
    adam = AdaMod()
    model.compile(loss='binary_crossentropy',
                  optimizer=adam,
                  metrics=['acc'])  # metrics.binary_accuracy
    return model


np.random.seed(0)
tf.set_random_seed(0)
sess = tf.Session(graph=tf.get_default_graph())
K.set_session(sess)

print('--------------Start training----------------{}'.format(datetime.datetime.now()))
# X_train, x_val = copy.deepcopy(data[train_index]), copy.deepcopy(data[test_index])
# y_train, y_val = copy.deepcopy(labels[train_index]), copy.deepcopy(labels[test_index])
# del data, labels

print('Number of positive and negative classes in training and validation set')
print(y_train.sum(axis=0))
print(y_val.sum(axis=0))

class_weight = get_class_weights(np.asarray(y_train, 'int32')[:, 1])

FOLDER = "{}/{}".format(MODEL_FOLDER, datetime.datetime.now().strftime("%Y-%m-%d %H-%M"))
if not os.path.exists(FOLDER):
    os.makedirs(FOLDER)
callbacks = TensorBoard(log_dir=FOLDER,
                        write_graph=True, write_grads=False, histogram_freq=0,
                        write_images=True, embeddings_freq=0, embeddings_layer_names='embedding_1',
                        embeddings_metadata=None)

checkpoint = ModelCheckpoint(FOLDER + '/model.{epoch:02d}-{val_acc:.2f}.hdf5',
                             verbose=0, monitor='val_acc',
                             save_best_only=True, mode='auto')
checkpoint = ModelCheckpoint(FOLDER + '/model.{epoch:02d}-{val_acc:.2f}.hdf5')
print("model fitting - Hierachical LSTM")

print(class_weight)
model = create_dl_model()

model.fit(X_train, np.asarray(y_train, 'int32'),
# model.fit(X_train, np.asarray(y_train[:, 1], 'int32'),
#           validation_data=(x_val, np.asarray(y_val, 'int32')),
          # validation_data=(x_val, np.asarray(y_val[:, 1], 'int32')),
          shuffle=True,
          nb_epoch=200, batch_size=32, verbose=0, class_weight=class_weight, callbacks=[ callbacks])

model.save(FOLDER + '/my_model.h5')
K.clear_session()

In [ ]:
from keras.models import load_model
from utils.metrics import find_optimal_cutoff, get_max_acc

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import roc_curve, auc

In [ ]:
model_name = 'logs/DL/2020-02-19 15-04-50000-35000/2020-02-19 16-29/my_model.h5'

print(model_name)
model = load_model(model_name,
                   custom_objects={'optimizer': AdaMod})
probas_ = model.predict(x_val)
y_test = y_val[:, 1].astype(int)
max_accuracy, thresh = get_max_acc(y_test, probas_[:, 1])
pred = [1 if m > thresh else 0 for m in probas_[:, 1]]
print('Fold {} accuracy = {:.2f}'.format(0+1, max_accuracy*100))
print(classification_report(y_test, pred, target_names=['Control', 'Depress']))

In [ ]:
i = 0
MODEL_FOLDER = "logs/{}/{}".format('DL', datetime.datetime.now().strftime("%Y-%m-%d %H-%M") + '-' + str(MAX_NB_WORDS) + '-' + str(INPUT_LENGTH))
for data_index in split_data.split(depress, 4):
    data_fold = depress[depress.userid.isin(data_index)].copy()

    print(data_index[:10])
    print(data_fold.head())

    combine, labels = combine_users(control, data_fold)
    inputs, labels = select_post(combine, labels, MAX_POSTS)
    del combine, data_fold

    # transform Y to categories
    print(labels.label.value_counts())
    labels = labels.label.values
    labels = to_categorical(np.asarray(labels))

    print('Input number: ', len(inputs))
    print('Label number: ', len(labels))

    # alltexts = np.hstack(np.array(inputs))
    alltexts = np.hstack(np.array(inputs).flatten())

    print('Tokenizing text')
    start = time.time()
    tknzr = TweetTokenizer(reduce_len=True)
    alltexts = [tknzr.tokenize(text.lower()) for text in alltexts]

    tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
    tokenizer.fit_on_texts(alltexts)
    word_index = tokenizer.word_index
    print('Total %s unique tokens.' % len(word_index))
    print('-----------{}-----------'.format(time.time() - start))

    print('Transforming text to input sequences')
    start = time.time()
    data = np.zeros((len(inputs), INPUT_LENGTH), dtype='int32')
    ppp = []
    for i, posts in enumerate(inputs):
        for j, post in enumerate(posts):
            pp = tokenizer.texts_to_sequences([post])
            if len(pp) > 18:
                ppp = ppp + pp[:18]
            else:
                ppp = ppp + pp
        sequences = tokenizer.texts_to_sequences([' '.join(posts)])
        seq_data = pad_sequences(sequences, maxlen=INPUT_LENGTH)
        data[i] = seq_data
#     del inputs, alltexts
    print('-----------{}-----------'.format(time.time() - start))
    
    for i, posts in enumerate(inputs):
        for j, post in enumerate(posts):
            if j < MAX_POSTS:
                sequences = tokenizer.texts_to_sequences([post])
                seq_data = pad_sequences(sequences, maxlen=MAX_POST_LENGTH)
                data[i, j] = seq_data
    elapsed_time_fl = (time.time() - start)
    print(elapsed_time_fl)

    print('Finished loading data')
    print('Shape of data tensor:', data.shape)
    print('Shape of label tensor:', labels.shape)

    num_words = min(MAX_NB_WORDS, len(word_index) + 1)

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=12345)
    for train_index, test_index in skf.split(np.asarray(data), labels[:, 1]):
        np.random.seed(0)
        tf.set_random_seed(0)
        sess = tf.Session(graph=tf.get_default_graph())
        K.set_session(sess)

        print('--------------Start training----------------{}'.format(datetime.datetime.now()))
        X_train, x_val = copy.deepcopy(data[train_index]), copy.deepcopy(data[test_index])
        y_train, y_val = copy.deepcopy(labels[train_index]), copy.deepcopy(labels[test_index])
        break
    break

In [ ]:
MODEL_FOLDER = "logs/{}/{}".format('DL', datetime.datetime.now().strftime("%Y-%m-%d %H-%M") + '-' + str(MAX_NB_WORDS) + '-' + str(INPUT_LENGTH))
for data_index in split_data.split(depress, 4):
    data_fold = depress[depress.userid.isin(data_index)].copy()

    print(data_index[:10])
    print(data_fold.head())

    combine, labels = combine_users(control, data_fold)
    inputs, labels = select_post(combine, labels, MAX_POSTS)
    break

In [ ]:
data = np.zeros((len(inputs), INPUT_LENGTH), dtype='int32')
for i, posts in enumerate(inputs[:2]):
    ppp = []
    for j, post in enumerate(posts):
        pp = tokenizer.texts_to_sequences([post])
        if len(pp[0]) > 18:
            ppp = ppp + pp[0][-18:]
        else:
            ppp = ppp + pp[0]
    sequences = [ppp]
    seq_data = pad_sequences(sequences, maxlen=INPUT_LENGTH)
    data[i] = seq_data
#     del inputs, alltexts
print('-----------{}-----------'.format(time.time() - start))

In [ ]:
data2 = np.zeros((len(inputs), INPUT_LENGTH), dtype='int32')
for i, posts in enumerate(inputs):
    sequences = tokenizer.texts_to_sequences([' '.join(posts)])
    seq_data = pad_sequences(sequences, maxlen=INPUT_LENGTH)
    data2[i] = seq_data

In [ ]:
data2[:5]

In [ ]:
data[:5]

In [ ]:
import os
import time
import copy
import datetime
import numpy as np
import pandas as pd
from numpy.random import seed
from utils import split_data
from utils.tokeniser import Tokenizer
from utils.utils import get_class_weights
from utils.embed import load_glove_tw_vectors
from utils.users import load_users, select_post, combine_users
from sklearn.model_selection import StratifiedKFold
from nltk.tokenize import TweetTokenizer
from keras.optimizers import Adam, AdaMod
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Conv1D, GlobalMaxPooling1D
from keras.layers import Embedding, Dropout, Input
from keras.models import Model
from tensorflow import set_random_seed
import tensorflow as tf
from keras import backend as K


def create_dl_model():
    num_filters = 250
    drop = 0.5

    embedding_layer = Embedding(num_words,
                                EMBEDDING_DIM,
                                input_length=INPUT_LENGTH,
                                weights=[embedding_matrix],
                                trainable=False)
                                # trainable=True)

    model_input = Input(shape=(INPUT_LENGTH,), dtype='int32')
    model_layer = embedding_layer(model_input)
    model_layer = Dropout(drop)(model_layer)
    model_layer = Conv1D(filters=num_filters, kernel_size=3)(model_layer)
    model_layer = GlobalMaxPooling1D()(model_layer)
    model_layer = Dense(num_filters, activation='relu')(model_layer)
    model_layer = Dropout(drop)(model_layer)
    model_layer = Dense(2, activation='softmax')(model_layer)
    # model_layer = Dense(1, activation='sigmoid')(model_layer)
    model = Model(model_input, model_layer)

    print(model.summary())
#     adam = Adam(lr=0.007, clipvalue=7)
    adam = AdaMod()
    model.compile(loss='binary_crossentropy',
                  optimizer=adam,
                  metrics=['acc'])  # metrics.binary_accuracy
    return model


SEED = 1
seed(SEED)
set_random_seed(SEED)

POST_SIZE = 100
MAX_POSTS = 3200
MAX_NB_WORDS = 50000
INPUT_LENGTH = 35000
EMBEDDING_DIM = 100
NB_DEPRESS = 1984

DIR = '../data/'

inputs, labels = load_users(DIR, POST_SIZE, MAX_POSTS, NB_DEPRESS, True)
control = inputs[inputs.label == 0].reset_index(drop=True).copy()
depress = inputs[inputs.label == 1].reset_index(drop=True).copy()
del inputs, labels

embeddings_index = load_glove_tw_vectors(EMBEDDING_DIM)  # load pre-trained embedding vectors

i = 0
MODEL_FOLDER = "logs/{}/{}".format('DL', datetime.datetime.now().strftime("%Y-%m-%d %H-%M") + '-' + str(MAX_NB_WORDS) + '-' + str(INPUT_LENGTH))
for data_index in split_data.split(depress, 4):
    data_fold = depress[depress.userid.isin(data_index)].copy()

    print(data_index[:10])
    print(data_fold.head())

    combine, labels = combine_users(control, data_fold)
    inputs, labels = select_post(combine, labels, MAX_POSTS)
    del combine, data_fold

    # transform Y to categories
    print(labels.label.value_counts())
    labels = labels.label.values
    labels = to_categorical(np.asarray(labels))

    print('Input number: ', len(inputs))
    print('Label number: ', len(labels))

    # alltexts = np.hstack(np.array(inputs))
    alltexts = np.hstack(np.array(inputs).flatten())

    print('Tokenizing text')
    start = time.time()
    tknzr = TweetTokenizer(reduce_len=True)
    alltexts = [tknzr.tokenize(text.lower()) for text in alltexts]

    tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
    tokenizer.fit_on_texts(alltexts)
    word_index = tokenizer.word_index
    print('Total %s unique tokens.' % len(word_index))
    print('-----------{}-----------'.format(time.time() - start))

    print('Transforming text to input sequences')
#     start = time.time()
#     data = np.zeros((len(inputs), INPUT_LENGTH), dtype='int32')
#     for i, posts in enumerate(inputs):
#         sequences = tokenizer.texts_to_sequences([' '.join(posts)])
#         seq_data = pad_sequences(sequences, maxlen=INPUT_LENGTH, padding='pre', truncating='post')
#         data[i] = seq_data
#     del inputs, alltexts
#     print('-----------{}-----------'.format(time.time() - start))
    
    start = time.time()
    data = np.zeros((len(inputs), INPUT_LENGTH), dtype='int32')
    for i, posts in enumerate(inputs):
        ppp = []
        for j, post in enumerate(posts):
            pp = tokenizer.texts_to_sequences([post])
            if len(pp[0]) > 18:
                ppp = ppp + pp[0][:18]
            else:
                ppp = ppp + pp[0]
        sequences = [ppp]
        seq_data = pad_sequences(sequences, maxlen=INPUT_LENGTH, padding='pre', truncating='post')
        data[i] = seq_data
    del inputs, alltexts
    print('-----------{}-----------'.format(time.time() - start))

    print('Finished loading data')
    print('Shape of data tensor:', data.shape)
    print('Shape of label tensor:', labels.shape)

    num_words = min(MAX_NB_WORDS, len(word_index) + 1)

    print('Building embedding matrix')
    print('%s Selected word tokens' % num_words)

    start = time.time()
    embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
    # embedding_matrix = np.random.uniform(-1, 1, (num_words, EMBEDDING_DIM))
    for word, i in word_index.items():
        if i >= MAX_NB_WORDS:
            break
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    print('-----------{}-----------'.format(time.time() - start))

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=12345)
    for train_index, test_index in skf.split(np.asarray(data), labels[:, 1]):
        np.random.seed(0)
        tf.set_random_seed(0)
        sess = tf.Session(graph=tf.get_default_graph())
        K.set_session(sess)

        print('--------------Start training----------------{}'.format(datetime.datetime.now()))
        X_train, x_val = copy.deepcopy(data[train_index]), copy.deepcopy(data[test_index])
        y_train, y_val = copy.deepcopy(labels[train_index]), copy.deepcopy(labels[test_index])
#         del data, labels

        print('Number of positive and negative classes in training and validation set')
        print(y_train.sum(axis=0))
        print(y_val.sum(axis=0))

        class_weight = get_class_weights(np.asarray(y_train, 'int32')[:, 1])

        FOLDER = "{}/{}".format(MODEL_FOLDER, datetime.datetime.now().strftime("%Y-%m-%d %H-%M"))
        if not os.path.exists(FOLDER):
            os.makedirs(FOLDER)
        callbacks = TensorBoard(log_dir=FOLDER,
                                write_graph=True, write_grads=False, histogram_freq=0,
                                write_images=True, embeddings_freq=0, embeddings_layer_names='embedding_1',
                                embeddings_metadata=None)

        checkpoint = ModelCheckpoint(FOLDER + '/model.{epoch:02d}-{val_acc:.2f}.hdf5',
                                     verbose=0, monitor='val_acc',
                                     save_best_only=True, mode='auto')
        
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=15)

        print("model fitting - Hierachical LSTM")

        print(class_weight)
        model = create_dl_model()

        model.fit(X_train, np.asarray(y_train, 'int32'),
        # model.fit(X_train, np.asarray(y_train[:, 1], 'int32'),
                  validation_data=(x_val, np.asarray(y_val, 'int32')),
                  # validation_data=(x_val, np.asarray(y_val[:, 1], 'int32')),
                  shuffle=False,
                  nb_epoch=200, batch_size=32, verbose=0, class_weight=class_weight, callbacks=[checkpoint, callbacks, es])

        model.save(FOLDER + '/my_model.h5')
        del model, X_train, y_train, x_val, y_val, class_weight
        K.clear_session()
        print('--------------Finished training----------------'.format(datetime.datetime.now()))
        break


The number of control:  1699
The number of depress:  1984
Control shape:  (3944945, 3)
Depress shape:  (4996621, 3)
Selecting users with at least 100 posts
Control users:  1699
-----------12.117253303527832-----------
Depress users:  1984
-----------16.086098670959473-----------
Labelling
Loading embedding vectors from a file
Total 1193514 word vectors.
[ 780078428687065089            25567061           416684112
           608725707  912541969968066560          2700680921
          2329059770 1064934140007538688            43903078
            16656127]
                  tweetid   userid  \
3234  1143666788107866112  5882452   
3235  1143666924246560768  5882452   
3236  1143664692679720961  5882452   
3237  1143661172891066370  5882452   
3238  1143669737378893824  5882452   

                                                  clean  label  
3234  user user i've always felt like i was kind of ...      1  
3235  user user i cannot function without my morning...      1  
3236  user way 

C:\Users\Bright\.conda\envs\py_37\lib\site-packages\ipykernel_launcher.py:202: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


--------------Finished training----------------
[         2826065615          3960721393          4318344515
          3382208403          2599342730          2192916252
           181892265 1091181448843931649  836044275712475136
            60688549]
                  tweetid   userid  \
6479  1117414019164454912  6111852   
6480  1117414158188929024  6111852   
6481  1117413476631220226  6111852   
6482  1116951765906227200  6111852   
6483  1117037694268342272  6111852   

                                                  clean  label  
6479                     good morning, twitterverse url      1  
6480  rt user: everything you need to know about k-p...      1  
6481  it's also a rhetorical trick thwimp uses all t...      1  
6482                          rt user: this thread: url      1  
6483  putin has said america is no longer a leader. ...      1  
Selecting 3200 posts from each user
-----------18.413405656814575-----------
0    1699
1     496
Name: label, dtype: int64
Input

C:\Users\Bright\.conda\envs\py_37\lib\site-packages\ipykernel_launcher.py:202: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


--------------Finished training----------------
[           22936389          1586758399          3227968831
          2731028615          3056177975            25787341
 1104780414516977666           280932608            56891919
  800940814641856512]
               tweetid  userid  \
0   589962698353299457  802310   
1  1130666193289863168  802310   
2  1070479497898520577  802310   
3  1070482628153794560  802310   
4  1070483237879799809  802310   

                                               clean  label  
0  user github is giving me more grief than the l...      1  
1  user some of us just ride the risk-train and g...      1  
2  user super happy to see another competent deve...      1  
3  user you can't trust lnp to make sensible cybe...      1  
4  user user is user prepared to stand against <h...      1  
Selecting 3200 posts from each user
-----------18.52982783317566-----------
0    1699
1     496
Name: label, dtype: int64
Input number:  2195
Label number:  2195
Tokenizi

C:\Users\Bright\.conda\envs\py_37\lib\site-packages\ipykernel_launcher.py:202: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


--------------Finished training----------------
[ 960112188433338368 1004044855931293696           497519459
            41555588  753773198383128576          2868511826
 1019322256953106437  962874724987551744 1004689374221623297
          2237542326]
                   tweetid    userid  \
22499  1089551448323575809  13379712   
22500  1089551092256571393  13379712   
22501  1089551239837294594  13379712   
22502  1089551307801800709  13379712   
22503  1089551538832449537  13379712   

                                                   clean  label  
22499  rt user: cop: your neighbor has the word butth...      1  
22500  rt user: [prison]  prisoner: what's for breakf...      1  
22501  rt user: suspect: i ain’t talkin  cop: [sharpe...      1  
22502  rt user: [lighting a cigar] man i kicked my fa...      1  
22503  rt user: me: do you wanna go skydiving  her: i...      1  
Selecting 3200 posts from each user
-----------15.451961278915405-----------
0    1699
1     496
Name: label, 

C:\Users\Bright\.conda\envs\py_37\lib\site-packages\ipykernel_launcher.py:202: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


--------------Finished training----------------


In [ ]:
import os
import time
import copy
import datetime
import numpy as np
import pandas as pd
from numpy.random import seed
from utils import split_data
from utils.tokeniser import Tokenizer
from utils.utils import get_class_weights
from utils.embed import load_glove_tw_vectors
from utils.users import load_users, select_post, combine_users
from sklearn.model_selection import StratifiedKFold
from nltk.tokenize import TweetTokenizer
from keras.optimizers import Adam, AdaMod
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Conv1D, GlobalMaxPooling1D
from keras.layers import Embedding, Dropout, Input
from keras.models import Model
from tensorflow import set_random_seed
import tensorflow as tf
from keras import backend as K


def create_dl_model():
    num_filters = 250
    drop = 0.5

    embedding_layer = Embedding(num_words,
                                EMBEDDING_DIM,
                                input_length=INPUT_LENGTH,
                                weights=[embedding_matrix],
                                trainable=False)
                                # trainable=True)

    model_input = Input(shape=(INPUT_LENGTH,), dtype='int32')
    model_layer = embedding_layer(model_input)
    model_layer = Dropout(drop)(model_layer)
    model_layer = Conv1D(filters=num_filters, kernel_size=3)(model_layer)
    model_layer = GlobalMaxPooling1D()(model_layer)
    model_layer = Dense(num_filters, activation='relu')(model_layer)
    model_layer = Dropout(drop)(model_layer)
    model_layer = Dense(2, activation='softmax')(model_layer)
    # model_layer = Dense(1, activation='sigmoid')(model_layer)
    model = Model(model_input, model_layer)

    print(model.summary())
#     adam = Adam(lr=0.007, clipvalue=7)
    adam = AdaMod()
    model.compile(loss='binary_crossentropy',
                  optimizer=adam,
                  metrics=['acc'])  # metrics.binary_accuracy
    return model


SEED = 1
seed(SEED)
set_random_seed(SEED)

POST_SIZE = 100
MAX_POSTS = 3200
MAX_NB_WORDS = 50000
INPUT_LENGTH = 35000
EMBEDDING_DIM = 100
NB_DEPRESS = 1984

DIR = '../data/'

inputs, labels = load_users(DIR, POST_SIZE, MAX_POSTS, NB_DEPRESS, True)
control = inputs[inputs.label == 0].reset_index(drop=True).copy()
depress = inputs[inputs.label == 1].reset_index(drop=True).copy()
del inputs, labels

embeddings_index = load_glove_tw_vectors(EMBEDDING_DIM)  # load pre-trained embedding vectors

fold = 0
MODEL_FOLDER = "logs/{}/{}".format('DL', datetime.datetime.now().strftime("%Y-%m-%d %H-%M") + '-' + str(MAX_NB_WORDS) + '-' + str(INPUT_LENGTH))
for data_index in split_data.split(depress, 4):
    data_fold = depress[depress.userid.isin(data_index)].copy()

    print(data_index[:10])
    print(data_fold.head())

    combine, labels = combine_users(control, data_fold)
    inputs, labels = select_post(combine, labels, MAX_POSTS)
    del combine, data_fold

    # transform Y to categories
    print(labels.label.value_counts())
    labels = labels.label.values
    labels = to_categorical(np.asarray(labels))

    print('Input number: ', len(inputs))
    print('Label number: ', len(labels))

    # alltexts = np.hstack(np.array(inputs))
    alltexts = np.hstack(np.array(inputs).flatten())

    print('Tokenizing text')
    start = time.time()
    tknzr = TweetTokenizer(reduce_len=True)
    alltexts = [tknzr.tokenize(text.lower()) for text in alltexts]

    tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
    tokenizer.fit_on_texts(alltexts)
    word_index = tokenizer.word_index
    print('Total %s unique tokens.' % len(word_index))
    print('-----------{}-----------'.format(time.time() - start))

    print('Transforming text to input sequences')
    start = time.time()
    data = np.zeros((len(inputs), INPUT_LENGTH), dtype='int32')
    for i, posts in enumerate(inputs):
        sequences = tokenizer.texts_to_sequences([' '.join(posts)])
        seq_data = pad_sequences(sequences, maxlen=INPUT_LENGTH, padding='pre', truncating='post')
        data[i] = seq_data
    del inputs, alltexts
    print('-----------{}-----------'.format(time.time() - start))
    
#     start = time.time()
#     data = np.zeros((len(inputs), INPUT_LENGTH), dtype='int32')
#     for i, posts in enumerate(inputs):
#         ppp = []
#         for j, post in enumerate(posts):
#             pp = tokenizer.texts_to_sequences([post])
#             if len(pp[0]) > 18:
#                 ppp = ppp + pp[0][:18]
#             else:
#                 ppp = ppp + pp[0]
#         sequences = [ppp]
#         seq_data = pad_sequences(sequences, maxlen=INPUT_LENGTH, padding='pre', truncating='post')
#         data[i] = seq_data
#     del inputs, alltexts
#     print('-----------{}-----------'.format(time.time() - start))

    print('Finished loading data')
    print('Shape of data tensor:', data.shape)
    print('Shape of label tensor:', labels.shape)

    num_words = min(MAX_NB_WORDS, len(word_index) + 1)

    print('Building embedding matrix')
    print('%s Selected word tokens' % num_words)

    start = time.time()
    embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
    # embedding_matrix = np.random.uniform(-1, 1, (num_words, EMBEDDING_DIM))
    for word, i in word_index.items():
        if i >= MAX_NB_WORDS:
            break
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    print('-----------{}-----------'.format(time.time() - start))

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=12345)
    for train_index, test_index in skf.split(np.asarray(data), labels[:, 1]):
        np.random.seed(0)
        tf.set_random_seed(0)
        sess = tf.Session(graph=tf.get_default_graph())
        K.set_session(sess)

        print('--------------Start training----------------{}'.format(datetime.datetime.now()))
        X_train, x_val = copy.deepcopy(data[train_index]), copy.deepcopy(data[test_index])
        y_train, y_val = copy.deepcopy(labels[train_index]), copy.deepcopy(labels[test_index])
#         del data, labels

        print('Number of positive and negative classes in training and validation set')
        print(y_train.sum(axis=0))
        print(y_val.sum(axis=0))

        class_weight = get_class_weights(np.asarray(y_train, 'int32')[:, 1])

        FOLDER = "{}/{}".format(MODEL_FOLDER, datetime.datetime.now().strftime("%Y-%m-%d %H-%M"))
        if not os.path.exists(FOLDER):
            os.makedirs(FOLDER)
        callbacks = TensorBoard(log_dir=FOLDER,
                                write_graph=True, write_grads=False, histogram_freq=0,
                                write_images=True, embeddings_freq=0, embeddings_layer_names='embedding_1',
                                embeddings_metadata=None)

        checkpoint = ModelCheckpoint(FOLDER + '/model.{epoch:02d}-{val_acc:.2f}.hdf5',
                                     verbose=0, monitor='val_acc',
                                     save_best_only=True, mode='auto')
        
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=15)

        print("model fitting - Hierachical LSTM")

        print(class_weight)
        model = create_dl_model()

        model.fit(X_train, np.asarray(y_train, 'int32'),
        # model.fit(X_train, np.asarray(y_train[:, 1], 'int32'),
                  validation_data=(x_val, np.asarray(y_val, 'int32')),
                  # validation_data=(x_val, np.asarray(y_val[:, 1], 'int32')),
                  shuffle=False,
                  nb_epoch=200, batch_size=32, verbose=0, class_weight=class_weight, callbacks=[checkpoint, callbacks, es])
        
        probas_ = model.predict(x_val)
        df = pd.DataFrame(np.asarray(y_val[:, 1], 'int32'), columns=['label'])
        df['class0'] = probas_[:, 0]
        df['class1'] = probas_[:, 1]
        df.to_csv('dl-' + str(INPUT_LENGTH) + '-' + str(EMBEDDING_DIM) + str(fold) + '.csv', index=False)
        
        model.save(FOLDER + '/my_model.h5')
        del model, X_train, y_train, x_val, y_val, class_weight
        fold += 1
        K.clear_session()
        print('--------------Finished training----------------'.format(datetime.datetime.now()))
        break


The number of control:  1699
The number of depress:  1984
Control shape:  (3944945, 3)
Depress shape:  (4996621, 3)
Selecting users with at least 100 posts
Control users:  1699
-----------12.201915979385376-----------
Depress users:  1984
-----------16.185059785842896-----------
Labelling
Loading embedding vectors from a file
Total 1193514 word vectors.
[ 780078428687065089            25567061           416684112
           608725707  912541969968066560          2700680921
          2329059770 1064934140007538688            43903078
            16656127]
                  tweetid   userid  \
3234  1143666788107866112  5882452   
3235  1143666924246560768  5882452   
3236  1143664692679720961  5882452   
3237  1143661172891066370  5882452   
3238  1143669737378893824  5882452   

                                                  clean  label  
3234  user user i've always felt like i was kind of ...      1  
3235  user user i cannot function without my morning...      1  
3236  user way 

C:\Users\Bright\.conda\envs\py_37\lib\site-packages\ipykernel_launcher.py:202: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


--------------Finished training----------------
[         2826065615          3960721393          4318344515
          3382208403          2599342730          2192916252
           181892265 1091181448843931649  836044275712475136
            60688549]
                  tweetid   userid  \
6479  1117414019164454912  6111852   
6480  1117414158188929024  6111852   
6481  1117413476631220226  6111852   
6482  1116951765906227200  6111852   
6483  1117037694268342272  6111852   

                                                  clean  label  
6479                     good morning, twitterverse url      1  
6480  rt user: everything you need to know about k-p...      1  
6481  it's also a rhetorical trick thwimp uses all t...      1  
6482                          rt user: this thread: url      1  
6483  putin has said america is no longer a leader. ...      1  
Selecting 3200 posts from each user
-----------16.286903381347656-----------
0    1699
1     496
Name: label, dtype: int64
Input

C:\Users\Bright\.conda\envs\py_37\lib\site-packages\ipykernel_launcher.py:202: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


--------------Finished training----------------
[           22936389          1586758399          3227968831
          2731028615          3056177975            25787341
 1104780414516977666           280932608            56891919
  800940814641856512]
               tweetid  userid  \
0   589962698353299457  802310   
1  1130666193289863168  802310   
2  1070479497898520577  802310   
3  1070482628153794560  802310   
4  1070483237879799809  802310   

                                               clean  label  
0  user github is giving me more grief than the l...      1  
1  user some of us just ride the risk-train and g...      1  
2  user super happy to see another competent deve...      1  
3  user you can't trust lnp to make sensible cybe...      1  
4  user user is user prepared to stand against <h...      1  
Selecting 3200 posts from each user
-----------15.779103994369507-----------
0    1699
1     496
Name: label, dtype: int64
Input number:  2195
Label number:  2195
Tokeniz

C:\Users\Bright\.conda\envs\py_37\lib\site-packages\ipykernel_launcher.py:202: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


--------------Finished training----------------
[ 960112188433338368 1004044855931293696           497519459
            41555588  753773198383128576          2868511826
 1019322256953106437  962874724987551744 1004689374221623297
          2237542326]
                   tweetid    userid  \
22499  1089551448323575809  13379712   
22500  1089551092256571393  13379712   
22501  1089551239837294594  13379712   
22502  1089551307801800709  13379712   
22503  1089551538832449537  13379712   

                                                   clean  label  
22499  rt user: cop: your neighbor has the word butth...      1  
22500  rt user: [prison]  prisoner: what's for breakf...      1  
22501  rt user: suspect: i ain’t talkin  cop: [sharpe...      1  
22502  rt user: [lighting a cigar] man i kicked my fa...      1  
22503  rt user: me: do you wanna go skydiving  her: i...      1  
Selecting 3200 posts from each user
-----------15.695070266723633-----------
0    1699
1     496
Name: label, 

C:\Users\Bright\.conda\envs\py_37\lib\site-packages\ipykernel_launcher.py:202: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


--------------Finished training----------------


In [ ]:
import os
import sys
import time
import pickle
import datetime
import pandas as pd
import numpy as np
from numpy.random import seed
from utils import split_data
from attention import AttentionWithContext
from utils.users import load_users, select_post, combine_users
from utils.utils import get_class_weights
from utils.embed import load_glove_tw_vectors
from utils.tokeniser import Tokenizer
from sklearn.model_selection import StratifiedKFold
from nltk.tokenize import TweetTokenizer
from keras.optimizers import Adam, AdaMod
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.models import Model
from keras.layers import Dense, Input
from keras.layers import Embedding, Dropout, Bidirectional, TimeDistributed
from keras.layers import CuDNNGRU, CuDNNLSTM
from keras.layers import BatchNormalization
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras import backend as K
import tensorflow as tf
from tensorflow import set_random_seed
from keras.initializers import Constant


def create_atte_model():
    embedding_layer = Embedding(num_words,
                                EMBEDDING_DIM,
                                input_length=MAX_POST_LENGTH,
                                weights=[embedding_matrix],
                                trainable=False)

    sequence_input = Input(shape=(MAX_POST_LENGTH,))
    embedded_sequences = embedding_layer(sequence_input)
    l_lstm_sent = Bidirectional(CuDNNGRU(50, return_sequences=True))(embedded_sequences)
    l_lstm_sent = Dropout(0.5)(l_lstm_sent)
    l_lstm_sent = AttentionWithContext()(l_lstm_sent)
    l_lstm_sent = Dropout(0.5)(l_lstm_sent)
    preds = Dense(units=2, activation='softmax')(l_lstm_sent)

    sentEncoder = Model(sequence_input, preds)

    print(sentEncoder.summary())

    review_input = Input(shape=(MAX_POSTS, MAX_POST_LENGTH))
    l_lstm_sent = TimeDistributed(sentEncoder)(review_input)
    l_lstm_sent = BatchNormalization()(l_lstm_sent)
    l_lstm_sent = Dropout(0.5)(l_lstm_sent)
    l_lstm_sent = Bidirectional(CuDNNGRU(4, return_sequences=True))(l_lstm_sent)
    l_lstm_sent = Dropout(0.5)(l_lstm_sent)
    l_lstm_sent = AttentionWithContext()(l_lstm_sent)
    l_lstm_sent = Dropout(0.5)(l_lstm_sent)
    preds = Dense(2, activation='softmax')(l_lstm_sent)
    model = Model(review_input, preds)

    print(model.summary())

    adam = AdaMod()

    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['acc'])

    return model


num_words = 50000
EMBEDDING_DIM = 100
MAX_POST_LENGTH = 18
MAX_POSTS = 2000
embedding_matrix = np.zeros((50000, 100))
create_atte_model()

Using TensorFlow backend.
C:\Users\Bright\.conda\envs\py_37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Bright\.conda\envs\py_37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Bright\.conda\envs\py_37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Bright\.conda\envs\py_37\lib\site-packages\t






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 18)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 18, 100)           5000000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 18, 100)           45600     
_________________________________________________________________
dropout_1 (Dropout)          (None, 18, 100)           0         
_________________________________________________________________
attention_with_context_1 (At (None, 100)               10200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
______________________

In [ ]:
from keras.layers import concatenate
def create_ana_model():
    embedding_layer = Embedding(num_words,
                                EMBEDDING_DIM,
                                input_length=MAX_POST_LENGTH,
                                weights=[embedding_matrix],
                                trainable=False)

    sequence_input = Input(shape=(MAX_POST_LENGTH,))
    embedded_sequences = embedding_layer(sequence_input)
    l_lstm_sent = Bidirectional(CuDNNGRU(50, return_sequences=True))(embedded_sequences)
    l_lstm_sent = Dropout(0.5)(l_lstm_sent)
    l_lstm_sent = AttentionWithContext()(l_lstm_sent)
    l_lstm_sent = Dropout(0.5)(l_lstm_sent)
    preds = Dense(units=2, activation='softmax')(l_lstm_sent)
    sentEncoder = Model(sequence_input, preds)
    print(sentEncoder.summary())

    ana_input = Input(shape=(MAX_POSTS, len([1,2,3,4,5,6])))

    review_input = Input(shape=(MAX_POSTS, MAX_POST_LENGTH))
    l_lstm_sent = TimeDistributed(sentEncoder)(review_input)
    l_lstm_sent = concatenate([l_lstm_sent, ana_input])
    l_lstm_sent = BatchNormalization()(l_lstm_sent)
    l_lstm_sent = Dropout(0.5)(l_lstm_sent)
    l_lstm_sent = Bidirectional(CuDNNGRU(16, return_sequences=True))(l_lstm_sent)
    l_lstm_sent = Dropout(0.5)(l_lstm_sent)
    l_lstm_sent = AttentionWithContext()(l_lstm_sent)
    l_lstm_sent = Dropout(0.5)(l_lstm_sent)
    preds = Dense(2, activation='softmax')(l_lstm_sent)
    model = Model([review_input, ana_input], preds)
    print(model.summary())

    adam = AdaMod()
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['acc'])

    return model

create_ana_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 18)                0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 18, 100)           5000000   
_________________________________________________________________
bidirectional_6 (Bidirection (None, 18, 100)           45600     
_________________________________________________________________
dropout_13 (Dropout)         (None, 18, 100)           0         
_________________________________________________________________
attention_with_context_6 (At (None, 100)               10200     
_________________________________________________________________
dropout_14 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 202       
Total para

In [ ]:
import os
import time
import copy
import datetime
import numpy as np
import pandas as pd
from numpy.random import seed
from utils import split_data
from utils.tokeniser import Tokenizer
from utils.utils import get_class_weights
from utils.embed import load_glove_tw_vectors
from utils.users import load_users, select_post, combine_users
from sklearn.model_selection import StratifiedKFold
from nltk.tokenize import TweetTokenizer
from keras.optimizers import Adam, AdaMod
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Conv1D, GlobalMaxPooling1D
from keras.layers import Embedding, Dropout, Input
from keras.models import Model
from tensorflow import set_random_seed
import tensorflow as tf
from keras import backend as K
from keras.models import load_model
from utils.metrics import get_max_acc, find_optimal_cutoff
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import glob


def create_dl_model():
    num_filters = 250
    drop = 0.5

    embedding_layer = Embedding(num_words,
                                EMBEDDING_DIM,
                                input_length=INPUT_LENGTH,
                                weights=[embedding_matrix],
                                trainable=False)
                                # trainable=True)

    model_input = Input(shape=(INPUT_LENGTH,), dtype='int32')
    model_layer = embedding_layer(model_input)
    model_layer = Dropout(drop)(model_layer)
    model_layer = Conv1D(filters=num_filters, kernel_size=3)(model_layer)
    model_layer = GlobalMaxPooling1D()(model_layer)
    # model_layer = Dropout(drop)(model_layer)
    model_layer = Dense(num_filters, activation='relu')(model_layer)
    model_layer = Dropout(drop)(model_layer)
    # model_layer = Dense(2, activation='softmax')(model_layer)
    model_layer = Dense(1, activation='sigmoid')(model_layer)
    model = Model(model_input, model_layer)

    print(model.summary())
    adam = Adam(lr=0.002)
    # adam = AdaMod()
    model.compile(loss='binary_crossentropy',
                  optimizer=adam,
                  metrics=['acc'])  # metrics.binary_accuracy
    return model


SEED = 1
seed(SEED)
set_random_seed(SEED)

POST_SIZE = 100
MAX_POSTS = 3200
MAX_NB_WORDS = 50000
INPUT_LENGTH = 35000
EMBEDDING_DIM = 100
NB_DEPRESS = 1984

DIR = '../data/'

inputs, labels = load_users(DIR, POST_SIZE, MAX_POSTS, NB_DEPRESS, True)
control = inputs[inputs.label == 0].reset_index(drop=True).copy()
depress = inputs[inputs.label == 1].reset_index(drop=True).copy()
del inputs, labels

embeddings_index = load_glove_tw_vectors(EMBEDDING_DIM)  # load pre-trained embedding vectors

i = 0
fold = 0
results = []
MODEL_FOLDER = "logs/{}/{}".format('DL', datetime.datetime.now().strftime("%Y-%m-%d %H-%M") + '-' + str(MAX_NB_WORDS) + '-' + str(INPUT_LENGTH))
for data_index in split_data.split(depress, 4):
    data_fold = depress[depress.userid.isin(data_index)].copy()

    print(data_index[:10])
    print(data_fold.head())

    combine, labels = combine_users(control, data_fold)
    inputs, labels = select_post(combine, labels, MAX_POSTS)
    del combine, data_fold

    # transform Y to categories
    print(labels.label.value_counts())
    labels = labels.label.values
    labels = to_categorical(np.asarray(labels))

    print('Input number: ', len(inputs))
    print('Label number: ', len(labels))

    # alltexts = np.hstack(np.array(inputs))
    alltexts = np.hstack(np.array(inputs).flatten())

    print('Tokenizing text')
    start = time.time()
    tknzr = TweetTokenizer(reduce_len=True)
    alltexts = [tknzr.tokenize(text.lower()) for text in alltexts]

    tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
    tokenizer.fit_on_texts(alltexts)
    word_index = tokenizer.word_index
    print('Total %s unique tokens.' % len(word_index))
    print('-----------{}-----------'.format(time.time() - start))

    print('Transforming text to input sequences')
    start = time.time()
    data = np.zeros((len(inputs), INPUT_LENGTH), dtype='int32')
    for i, posts in enumerate(inputs):
        sequences = tokenizer.texts_to_sequences([' '.join(posts)])
        # sequences = tokenizer.lists_to_sequences([np.hstack(np.array(posts))])
        seq_data = pad_sequences(sequences, maxlen=INPUT_LENGTH, padding='pre', truncating='post')
        data[i] = seq_data
    del inputs, alltexts
    print('-----------{}-----------'.format(time.time() - start))

    print('Finished loading data')
    print('Shape of data tensor:', data.shape)
    print('Shape of label tensor:', labels.shape)

    num_words = min(MAX_NB_WORDS, len(word_index) + 1)

    print('Building embedding matrix')
    print('%s Selected word tokens' % num_words)

    if 'embeddings_index' in globals():
        start = time.time()
        embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
        # embedding_matrix = np.random.uniform(-1, 1, (num_words, EMBEDDING_DIM))
        for word, i in word_index.items():
            if i >= MAX_NB_WORDS:
                break
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                # words not found in embedding index will be all-zeros.
                embedding_matrix[i] = embedding_vector
        print('-----------{}-----------'.format(time.time() - start))

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=12345)
    for train_index, test_index in skf.split(np.asarray(data), labels[:, 1]):
        np.random.seed(0)
        tf.set_random_seed(0)
        sess = tf.Session(graph=tf.get_default_graph())
        K.set_session(sess)

        print('--------------Start training----------------{}'.format(datetime.datetime.now()))
        X_train, x_val = copy.deepcopy(data[train_index]), copy.deepcopy(data[test_index])
        y_train, y_val = copy.deepcopy(labels[train_index]), copy.deepcopy(labels[test_index])
        del data, labels

        print('Number of positive and negative classes in training and validation set')
        print(y_train.sum(axis=0))
        print(y_val.sum(axis=0))

        class_weight = get_class_weights(np.asarray(y_train, 'int32')[:, 1])

        FOLDER = "{}/{}".format(MODEL_FOLDER, datetime.datetime.now().strftime("%Y-%m-%d %H-%M"))
        if not os.path.exists(FOLDER):
            os.makedirs(FOLDER)
        callbacks = TensorBoard(log_dir=FOLDER,
                                write_graph=True, write_grads=False, histogram_freq=0,
                                write_images=True, embeddings_freq=0, embeddings_layer_names='embedding_1',
                                embeddings_metadata=None)

        checkpoint = ModelCheckpoint(FOLDER + '/model.{epoch:02d}-{val_acc:.2f}.hdf5',
                                     verbose=0, monitor='val_acc',
                                     save_best_only=True, mode='auto')

        es = EarlyStopping(monitor='val_acc', mode='auto', verbose=0, patience=10)

        print("model fitting - Hierachical LSTM")

        print(class_weight)
        model = create_dl_model()

        # model.fit(X_train, np.asarray(y_train, 'int32'),
        model.fit(X_train, np.asarray(y_train[:, 1], 'int32'),
                  # validation_data=(x_val, np.asarray(y_val, 'int32')),
                  validation_data=(x_val, np.asarray(y_val[:, 1], 'int32')),
                  shuffle=False,
                  nb_epoch=200, batch_size=32, verbose=0, class_weight=class_weight,
                  callbacks=[checkpoint, callbacks, es])

        model.save(FOLDER + '/my_model.h5')

        print('--------------Start eva----------------'.format(datetime.datetime.now()))
        model_name = sorted(glob.iglob(os.path.join(FOLDER, '*.h5')), key=os.path.getctime, reverse=True)[0]
        model = load_model(model_name,
                           custom_objects={'optimizer': AdaMod})
        probas_ = model.predict(x_val)
        y_test = y_val[:, 1].astype(int)
#         max_accuracy, thresh = get_max_acc(y_test, probas_[:, 1])
#         pred = [1 if m > thresh else 0 for m in probas_[:, 1]]
        max_accuracy, thresh = get_max_acc(y_test, probas_)
        pred = [1 if m > thresh else 0 for m in probas_]
        print('Fold {} accuracy = {:.2f}'.format(fold + 1, max_accuracy * 100))
        print(classification_report(y_test, pred, target_names=['Control', 'Depress']))

        df = pd.DataFrame(y_test, columns=['label'])
#         df['class0'] = probas_[:, 0]
        df['class1'] = probas_
        df.to_csv(MODEL_FOLDER + '/dl-' + str(INPUT_LENGTH) + '-' + str(EMBEDDING_DIM) + str(fold) + '.csv', index=False)

        weighted = precision_recall_fscore_support(y_test, pred, average='weighted')
        macro = precision_recall_fscore_support(y_test, pred, average='macro')
        micro = precision_recall_fscore_support(y_test, pred, average='micro')
        both = precision_recall_fscore_support(y_test, pred)
        results.append(['dl-max-' + str(fold),
                        max_accuracy, weighted[0], weighted[1], weighted[2],
                        macro[0], macro[1], macro[2], micro[0], micro[1], micro[2],
                        both[0][0], both[1][0], both[2][0], both[0][1], both[1][1], both[2][1]])

        pred = [1 if m > 0.5 else 0 for m in probas_]
        print('Fold {} normal accuracy = {:.2f}'.format(fold + 1, accuracy_score(y_test, pred) * 100))
        max_accuracy = accuracy_score(y_test, pred)
        print(classification_report(y_test, pred, target_names=['Control', 'Depress']))

        weighted = precision_recall_fscore_support(y_test, pred, average='weighted')
        macro = precision_recall_fscore_support(y_test, pred, average='macro')
        micro = precision_recall_fscore_support(y_test, pred, average='micro')
        both = precision_recall_fscore_support(y_test, pred)
        results.append(['dl-nor-' + str(fold),
                        max_accuracy, weighted[0], weighted[1], weighted[2],
                        macro[0], macro[1], macro[2], micro[0], micro[1], micro[2],
                        both[0][0], both[1][0], both[2][0], both[0][1], both[1][1], both[2][1]])

        print('-------------------------------------------Start Best Model--------------------------------------------')

        model_name = sorted(glob.iglob(os.path.join(FOLDER, '*.hdf5')), key=os.path.getctime, reverse=True)[0]
        model = load_model(model_name,
                           custom_objects={'optimizer': AdaMod})
        probas_ = model.predict(x_val)
        y_test = y_val[:, 1].astype(int)
#         max_accuracy, thresh = get_max_acc(y_test, probas_[:, 1])
#         pred = [1 if m > thresh else 0 for m in probas_[:, 1]]
        max_accuracy, thresh = get_max_acc(y_test, probas_)
        pred = [1 if m > thresh else 0 for m in probas_]
        print('Fold {} accuracy = {:.2f}'.format(fold + 1, max_accuracy * 100))
        print(classification_report(y_test, pred, target_names=['Control', 'Depress']))

        df = pd.DataFrame(y_test, columns=['label'])
#         df['class0'] = probas_[:, 0]
        df['class1'] = probas_
        df.to_csv(MODEL_FOLDER + '/dl-best-' + str(INPUT_LENGTH) + '-' + str(EMBEDDING_DIM) + str(fold) + '.csv', index=False)

        weighted = precision_recall_fscore_support(y_test, pred, average='weighted')
        macro = precision_recall_fscore_support(y_test, pred, average='macro')
        micro = precision_recall_fscore_support(y_test, pred, average='micro')
        both = precision_recall_fscore_support(y_test, pred)
        results.append(['dl-best-max-' + str(fold),
                        max_accuracy, weighted[0], weighted[1], weighted[2],
                        macro[0], macro[1], macro[2], micro[0], micro[1], micro[2],
                        both[0][0], both[1][0], both[2][0], both[0][1], both[1][1], both[2][1]])

        pred = [1 if m > 0.5 else 0 for m in probas_]
        print('Fold {} normal accuracy = {:.2f}'.format(fold + 1, accuracy_score(y_test, pred) * 100))
        max_accuracy = accuracy_score(y_test, pred)
        print(classification_report(y_test, pred, target_names=['Control', 'Depress']))

        weighted = precision_recall_fscore_support(y_test, pred, average='weighted')
        macro = precision_recall_fscore_support(y_test, pred, average='macro')
        micro = precision_recall_fscore_support(y_test, pred, average='micro')
        both = precision_recall_fscore_support(y_test, pred)
        results.append(['dl-best-nor-' + str(fold),
                        max_accuracy, weighted[0], weighted[1], weighted[2],
                        macro[0], macro[1], macro[2], micro[0], micro[1], micro[2],
                        both[0][0], both[1][0], both[2][0], both[0][1], both[1][1], both[2][1]])

        del model, X_train, y_train, x_val, y_val, class_weight
        K.clear_session()
        fold += 1
        print('--------------Finished training----------------'.format(datetime.datetime.now()))
        break


Using TensorFlow backend.
C:\Users\Bright\.conda\envs\py_37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Bright\.conda\envs\py_37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Bright\.conda\envs\py_37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Bright\.conda\envs\py_37\lib\site-packages\t

The number of control:  1699
The number of depress:  1984
Control shape:  (3944945, 3)
Depress shape:  (4996621, 3)
Selecting users with at least 100 posts
Control users:  1699
-----------11.66613483428955-----------
Depress users:  1984
-----------15.844536304473877-----------
Labelling
Loading embedding vectors from a file
Total 1193514 word vectors.
[ 780078428687065089            25567061           416684112
           608725707  912541969968066560          2700680921
          2329059770 1064934140007538688            43903078
            16656127]
                  tweetid   userid  \
3234  1143666788107866112  5882452   
3235  1143666924246560768  5882452   
3236  1143664692679720961  5882452   
3237  1143661172891066370  5882452   
3238  1143669737378893824  5882452   

                                                  clean  label  
3234  user user i've always felt like i was kind of ...      1  
3235  user user i cannot function without my morning...      1  
3236  user way t

C:\Users\Bright\.conda\envs\py_37\lib\site-packages\ipykernel_launcher.py:197: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.




--------------Start eva----------------
Fold 1 accuracy = 93.39
              precision    recall  f1-score   support

     Control       0.94      0.98      0.96       340
     Depress       0.91      0.79      0.84        99

    accuracy                           0.93       439
   macro avg       0.92      0.88      0.90       439
weighted avg       0.93      0.93      0.93       439

Fold 1 normal accuracy = 67.43
              precision    recall  f1-score   support

     Control       1.00      0.58      0.73       340
     Depress       0.41      1.00      0.58        99

    accuracy                           0.67       439
   macro avg       0.70      0.79      0.66       439
weighted avg       0.87      0.67      0.70       439

-------------------------------------------Start Best Model--------------------------------------------
Fold 1 accuracy = 85.42
              precision    recall  f1-score   support

     Control       0.88      0.94      0.91       340
     Depress

C:\Users\Bright\.conda\envs\py_37\lib\site-packages\ipykernel_launcher.py:197: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


--------------Start eva----------------
Fold 2 accuracy = 91.80
              precision    recall  f1-score   support

     Control       0.92      0.98      0.95       340
     Depress       0.91      0.71      0.80        99

    accuracy                           0.92       439
   macro avg       0.91      0.84      0.87       439
weighted avg       0.92      0.92      0.91       439

Fold 2 normal accuracy = 70.62
              precision    recall  f1-score   support

     Control       0.99      0.63      0.77       340
     Depress       0.43      0.98      0.60        99

    accuracy                           0.71       439
   macro avg       0.71      0.80      0.68       439
weighted avg       0.86      0.71      0.73       439

-------------------------------------------Start Best Model--------------------------------------------
Fold 2 accuracy = 90.43
              precision    recall  f1-score   support

     Control       0.93      0.95      0.94       340
     Depress  

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Bright\.conda\envs\py_37\lib\site-packages\IPython\core\interactiveshell.py", line 3319, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-99bddbfae5c0>", line 125, in <module>
    sequences = tokenizer.texts_to_sequences([' '.join(posts)])
  File "C:\Users\Bright\PycharmProjects\final_thesis\utils\tokeniser.py", line 86, in texts_to_sequences
    seq.append(self.twk.tokenize(text.lower()))
  File "C:\Users\Bright\.conda\envs\py_37\lib\site-packages\nltk\tokenize\casual.py", line 301, in tokenize
    if not self.preserve_case:
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Bright\.conda\envs\py_37\lib\site-packages\IPython\core\interactiveshell.py", line 2034, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During ha

KeyboardInterrupt: 

In [ ]:
SEED = 1
seed(SEED)
set_random_seed(SEED)

POST_SIZE = 100
MAX_POSTS = 3200
MAX_NB_WORDS = 50000
INPUT_LENGTH = 35000
EMBEDDING_DIM = 100
NB_DEPRESS = 1984

DIR = '../data/'

inputs, labels = load_users(DIR, POST_SIZE, MAX_POSTS, NB_DEPRESS, True)
control = inputs[inputs.label == 0].reset_index(drop=True).copy()
depress = inputs[inputs.label == 1].reset_index(drop=True).copy()
del inputs, labels

i = 0
fold = 0
results = []
MODEL_FOLDER = "logs/{}/{}".format('DL', datetime.datetime.now().strftime("%Y-%m-%d %H-%M") + '-' + str(MAX_NB_WORDS) + '-' + str(INPUT_LENGTH))
for data_index in split_data.split(depress, 4):
    data_fold = depress[depress.userid.isin(data_index)].copy()

    print(data_index[:10])
    print(data_fold.head())

    combine, labels = combine_users(control, data_fold)
    inputs, labels = select_post(combine, labels, MAX_POSTS)
    del combine, data_fold

    # transform Y to categories
    print(labels.label.value_counts())
    labels = labels.label.values
    labels = to_categorical(np.asarray(labels))

    print('Input number: ', len(inputs))
    print('Label number: ', len(labels))

    # alltexts = np.hstack(np.array(inputs))
    alltexts = np.hstack(np.array(inputs).flatten())

    print('Tokenizing text')
    start = time.time()
    tknzr = TweetTokenizer(reduce_len=True)
    alltexts = [tknzr.tokenize(text.lower()) for text in alltexts]

    tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
    tokenizer.fit_on_texts(alltexts)
    word_index = tokenizer.word_index
    print('Total %s unique tokens.' % len(word_index))
    print('-----------{}-----------'.format(time.time() - start))

    print('Transforming text to input sequences')
    start = time.time()
    data = np.zeros((len(inputs), INPUT_LENGTH), dtype='int32')
    for i, posts in enumerate(inputs):
        sequences = tokenizer.texts_to_sequences([' '.join(posts)])
        seq_data = pad_sequences(sequences, maxlen=INPUT_LENGTH, padding='pre', truncating='post')
        data[i] = seq_data
    del inputs, alltexts
    print('-----------{}-----------'.format(time.time() - start))

    print('Finished loading data')
    print('Shape of data tensor:', data.shape)
    print('Shape of label tensor:', labels.shape)

    num_words = min(MAX_NB_WORDS, len(word_index) + 1)

    print('Building embedding matrix')
    print('%s Selected word tokens' % num_words)

    if 'embeddings_index' in globals():
        start = time.time()
        embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
        # embedding_matrix = np.random.uniform(-1, 1, (num_words, EMBEDDING_DIM))
        for word, i in word_index.items():
            if i >= MAX_NB_WORDS:
                break
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                # words not found in embedding index will be all-zeros.
                embedding_matrix[i] = embedding_vector
        print('-----------{}-----------'.format(time.time() - start))

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=12345)
    for train_index, test_index in skf.split(np.asarray(data), labels[:, 1]):
        np.random.seed(0)
        tf.set_random_seed(0)
        sess = tf.Session(graph=tf.get_default_graph())
        K.set_session(sess)

        print('--------------Start training----------------{}'.format(datetime.datetime.now()))
        X_train, x_val = copy.deepcopy(data[train_index]), copy.deepcopy(data[test_index])
        y_train, y_val = copy.deepcopy(labels[train_index]), copy.deepcopy(labels[test_index])
        break
    break

The number of control:  1699
The number of depress:  1984
Control shape:  (3944945, 3)
Depress shape:  (4996621, 3)
Selecting users with at least 100 posts
Control users:  1699
-----------13.731928825378418-----------
Depress users:  1984
-----------21.492454290390015-----------
Labelling
[ 780078428687065089            25567061           416684112
           608725707  912541969968066560          2700680921
          2329059770 1064934140007538688            43903078
            16656127]
                  tweetid   userid  \
3234  1143666788107866112  5882452   
3235  1143666924246560768  5882452   
3236  1143664692679720961  5882452   
3237  1143661172891066370  5882452   
3238  1143669737378893824  5882452   

                                                  clean  label  
3234  user user i've always felt like i was kind of ...      1  
3235  user user i cannot function without my morning...      1  
3236  user way too much of what you're describing so...      1  
3237  user betwe

In [ ]:
from keras.models import Sequential
from keras.layers import Activation

def create_dl_model():
    num_filters = 250
    drop = 0.2

    embedding_layer = Embedding(num_words,
                                EMBEDDING_DIM,
                                input_length=INPUT_LENGTH,
                                weights=[embedding_matrix],
                                trainable=False)
                                # trainable=True)

#     model_input = Input(shape=(INPUT_LENGTH,), dtype='int32')
#     model_layer = embedding_layer(model_input)
#     model_layer = Dropout(drop)(model_layer)
#     model_layer = Conv1D(filters=num_filters, kernel_size=3)(model_layer)
#     model_layer = GlobalMaxPooling1D()(model_layer)
#     model_layer = Dense(num_filters, activation='relu')(model_layer)
#     model_layer = Dropout(drop)(model_layer)
#     model_layer = Dense(2, activation='softmax')(model_layer)
#     model = Model(model_input, model_layer)
    
    model = Sequential()
    model.add(Embedding(num_words,
                        EMBEDDING_DIM,
                        input_length=INPUT_LENGTH,
                        weights=[embedding_matrix],
                                trainable=False))
#                         trainable=True))
    model.add(Dropout(drop))
    model.add(Conv1D(filters=num_filters, kernel_size=3))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(250))
    model.add(Activation('relu'))
    model.add(Dropout(drop))
    model.add(Dense(2))
    model.add(Activation('softmax'))

    print(model.summary())
    adam = Adam(lr=0.004, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    # adam = AdaMod()
    model.compile(loss='binary_crossentropy',
                  optimizer=adam,
                  metrics=['accuracy'])  # metrics.binary_accuracy
    return model

np.random.seed(0)
tf.set_random_seed(0)
sess = tf.Session(graph=tf.get_default_graph())
K.set_session(sess)

class_weight = get_class_weights(np.asarray(y_train, 'int32')[:, 1])

FOLDER = "{}/{}".format(MODEL_FOLDER, datetime.datetime.now().strftime("%Y-%m-%d %H-%M"))
if not os.path.exists(FOLDER):
    os.makedirs(FOLDER)
callbacks = TensorBoard(log_dir=FOLDER,
                        write_graph=True, write_grads=False, histogram_freq=0,
                        write_images=True, embeddings_freq=0, embeddings_layer_names='embedding_1',
                        embeddings_metadata=None)

checkpoint = ModelCheckpoint(FOLDER + '/model.{epoch:02d}-{val_acc:.2f}.hdf5',
                             verbose=0, monitor='val_acc',
                             save_best_only=True, mode='auto')

es = EarlyStopping(monitor='val_acc', mode='auto', verbose=0, patience=10)

print("model fitting - Hierachical LSTM")

print(class_weight)
model = create_dl_model()

model.fit(X_train, np.asarray(y_train, 'int32'),
# model.fit(X_train, np.asarray(y_train[:, 1], 'int32'),
          validation_data=(x_val, np.asarray(y_val, 'int32')),
#           validation_data=(x_val, np.asarray(y_val[:, 1], 'int32')),
          shuffle=False, #class_weight=class_weight,
          nb_epoch=200, batch_size=32, verbose=0, 
          callbacks=[checkpoint, callbacks, es])

model.save(FOLDER + '/my_model.h5')





print('--------------Start eva----------------'.format(datetime.datetime.now()))
model_name = sorted(glob.iglob(os.path.join(FOLDER, '*.h5')), key=os.path.getctime, reverse=True)[0]
model = load_model(model_name,
                   custom_objects={'optimizer': AdaMod})
probas_ = model.predict(x_val)
y_test = y_val[:, 1].astype(int)
max_accuracy, thresh = get_max_acc(y_test, probas_[:, 1])
pred = [1 if m > thresh else 0 for m in probas_[:, 1]]
print('Fold {} accuracy = {:.2f}'.format(fold + 1, max_accuracy * 100))
print(classification_report(y_test, pred, target_names=['Control', 'Depress']))

df = pd.DataFrame(y_test, columns=['label'])
df['class0'] = probas_[:, 0]
df['class1'] = probas_[:, 1]
df.to_csv(MODEL_FOLDER + '/dl-' + str(INPUT_LENGTH) + '-' + str(EMBEDDING_DIM) + str(fold) + '.csv', index=False)

weighted = precision_recall_fscore_support(y_test, pred, average='weighted')
macro = precision_recall_fscore_support(y_test, pred, average='macro')
micro = precision_recall_fscore_support(y_test, pred, average='micro')
both = precision_recall_fscore_support(y_test, pred)
results.append(['dl-max-' + str(fold),
                max_accuracy, weighted[0], weighted[1], weighted[2],
                macro[0], macro[1], macro[2], micro[0], micro[1], micro[2],
                both[0][0], both[1][0], both[2][0], both[0][1], both[1][1], both[2][1]])

pred = [1 if m > 0.5 else 0 for m in probas_[:, 1]]
print('Fold {} normal accuracy = {:.2f}'.format(fold + 1, accuracy_score(y_test, pred) * 100))
max_accuracy = accuracy_score(y_test, pred)
print(classification_report(y_test, pred, target_names=['Control', 'Depress']))

weighted = precision_recall_fscore_support(y_test, pred, average='weighted')
macro = precision_recall_fscore_support(y_test, pred, average='macro')
micro = precision_recall_fscore_support(y_test, pred, average='micro')
both = precision_recall_fscore_support(y_test, pred)
results.append(['dl-nor-' + str(fold),
                max_accuracy, weighted[0], weighted[1], weighted[2],
                macro[0], macro[1], macro[2], micro[0], micro[1], micro[2],
                both[0][0], both[1][0], both[2][0], both[0][1], both[1][1], both[2][1]])

print('-------------------------------------------Start Best Model--------------------------------------------')

model_name = sorted(glob.iglob(os.path.join(FOLDER, '*.hdf5')), key=os.path.getctime, reverse=True)[0]
model = load_model(model_name,
                   custom_objects={'optimizer': AdaMod})
probas_ = model.predict(x_val)
y_test = y_val[:, 1].astype(int)
max_accuracy, thresh = get_max_acc(y_test, probas_[:, 1])
pred = [1 if m > thresh else 0 for m in probas_[:, 1]]
print('Fold {} accuracy = {:.2f}'.format(fold + 1, max_accuracy * 100))
print(classification_report(y_test, pred, target_names=['Control', 'Depress']))

df = pd.DataFrame(y_test, columns=['label'])
df['class0'] = probas_[:, 0]
df['class1'] = probas_[:, 1]
df.to_csv(MODEL_FOLDER + '/dl-best-' + str(INPUT_LENGTH) + '-' + str(EMBEDDING_DIM) + str(fold) + '.csv', index=False)

weighted = precision_recall_fscore_support(y_test, pred, average='weighted')
macro = precision_recall_fscore_support(y_test, pred, average='macro')
micro = precision_recall_fscore_support(y_test, pred, average='micro')
both = precision_recall_fscore_support(y_test, pred)
results.append(['dl-best-max-' + str(fold),
                max_accuracy, weighted[0], weighted[1], weighted[2],
                macro[0], macro[1], macro[2], micro[0], micro[1], micro[2],
                both[0][0], both[1][0], both[2][0], both[0][1], both[1][1], both[2][1]])

pred = [1 if m > 0.5 else 0 for m in probas_[:, 1]]
print('Fold {} normal accuracy = {:.2f}'.format(fold + 1, accuracy_score(y_test, pred) * 100))
max_accuracy = accuracy_score(y_test, pred)
print(classification_report(y_test, pred, target_names=['Control', 'Depress']))

weighted = precision_recall_fscore_support(y_test, pred, average='weighted')
macro = precision_recall_fscore_support(y_test, pred, average='macro')
micro = precision_recall_fscore_support(y_test, pred, average='micro')
both = precision_recall_fscore_support(y_test, pred)
results.append(['dl-best-nor-' + str(fold),
                max_accuracy, weighted[0], weighted[1], weighted[2],
                macro[0], macro[1], macro[2], micro[0], micro[1], micro[2],
                both[0][0], both[1][0], both[2][0], both[0][1], both[1][1], both[2][1]])
K.clear_session()

model fitting - Hierachical LSTM
{0: 1.0, 1: 3.4231738035264483}
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 35000, 100)        5000000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 35000, 100)        0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 34998, 250)        75250     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 250)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 250)               62750     
_________________________________________________________________
activation_1 (Activation)    (None, 250)               0         
_________________________________________________________________
dropout_2 (

C:\Users\Bright\.conda\envs\py_37\lib\site-packages\ipykernel_launcher.py:78: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


--------------Start eva----------------
Fold 1 accuracy = 77.45
              precision    recall  f1-score   support

     Control       0.77      1.00      0.87       340
     Depress       0.00      0.00      0.00        99

    accuracy                           0.77       439
   macro avg       0.39      0.50      0.44       439
weighted avg       0.60      0.77      0.68       439

Fold 1 normal accuracy = 77.45
              precision    recall  f1-score   support

     Control       0.77      1.00      0.87       340
     Depress       0.00      0.00      0.00        99

    accuracy                           0.77       439
   macro avg       0.39      0.50      0.44       439
weighted avg       0.60      0.77      0.68       439

-------------------------------------------Start Best Model--------------------------------------------


C:\Users\Bright\.conda\envs\py_37\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fold 1 accuracy = 77.45
              precision    recall  f1-score   support

     Control       0.77      1.00      0.87       340
     Depress       0.00      0.00      0.00        99

    accuracy                           0.77       439
   macro avg       0.39      0.50      0.44       439
weighted avg       0.60      0.77      0.68       439

Fold 1 normal accuracy = 77.45
              precision    recall  f1-score   support

     Control       0.77      1.00      0.87       340
     Depress       0.00      0.00      0.00        99

    accuracy                           0.77       439
   macro avg       0.39      0.50      0.44       439
weighted avg       0.60      0.77      0.68       439



C:\Users\Bright\.conda\envs\py_37\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
def create_dl_model():
    num_filters = 250
    drop = 0.2

    embedding_layer = Embedding(num_words,
                                EMBEDDING_DIM,
                                input_length=INPUT_LENGTH,
                                weights=[embedding_matrix],
                                trainable=False)
                                # trainable=True)

#     model_input = Input(shape=(15000,), dtype='int32')
#     model_layer = embedding_layer(model_input)
#     model_layer = Dropout(drop)(model_layer)
#     model_layer = Conv1D(filters=num_filters, kernel_size=3)(model_layer)
#     model_layer = GlobalMaxPooling1D()(model_layer)
#     # model_layer = Dropout(drop)(model_layer)
#     model_layer = Dense(num_filters, activation='relu')(model_layer)
#     model_layer = Dropout(drop)(model_layer)
#     model_layer = Dense(1, activation='sigmoid')(model_layer)
#     model = Model(model_input, model_layer)
    model = Sequential()
    model.add(Embedding(num_words,
                        EMBEDDING_DIM,
                        input_length=15000,
                        weights=[embedding_matrix],
                                trainable=False))
#                         trainable=True))
    model.add(Dropout(drop))
    model.add(Conv1D(filters=num_filters, kernel_size=3))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(250))
    model.add(Activation('relu'))
    model.add(Dropout(drop))
#     model.add(Dense(1))
#     model.add(Activation('sigmoid'))
    model.add(Dense(2))
    model.add(Activation('softmax'))

    print(model.summary())
    adam = Adam(lr=0.004, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    # adam = AdaMod()
    model.compile(loss='binary_crossentropy',
                  optimizer=adam,
                  metrics=['acc'])  # metrics.binary_accuracy
    return model

np.random.seed(0)
tf.set_random_seed(0)
sess = tf.Session(graph=tf.get_default_graph())
K.set_session(sess)

class_weight = get_class_weights(np.asarray(y_train, 'int32')[:, 1])

FOLDER = "{}/{}".format(MODEL_FOLDER, datetime.datetime.now().strftime("%Y-%m-%d %H-%M"))
if not os.path.exists(FOLDER):
    os.makedirs(FOLDER)
callbacks = TensorBoard(log_dir=FOLDER,
                        write_graph=True, write_grads=False, histogram_freq=0,
                        write_images=True, embeddings_freq=0, embeddings_layer_names='embedding_1',
                        embeddings_metadata=None)

checkpoint = ModelCheckpoint(FOLDER + '/model.{epoch:02d}-{val_acc:.2f}.hdf5',
                             verbose=0, monitor='val_acc',
                             save_best_only=True, mode='auto')

es = EarlyStopping(monitor='val_loss', mode='auto', verbose=0, patience=10)

print("model fitting - Hierachical LSTM")

print(class_weight)
model = create_dl_model()

X_train2 = np.zeros((len(X_train), 15000), dtype='int32')
for i, posts in enumerate(X_train):
    seq_data = pad_sequences([posts], maxlen=15000, padding='pre', truncating='post')
    X_train2[i] = seq_data
    
x_val2 = np.zeros((len(x_val), 15000), dtype='int32')
for i, posts in enumerate(x_val):
    seq_data = pad_sequences([posts], maxlen=15000, padding='pre', truncating='post')
    x_val2[i] = seq_data

model.fit(X_train2, np.asarray(y_train, 'int32'),
# model.fit(X_train2, np.asarray(y_train[:, 1], 'int32'),
          validation_data=(x_val2, np.asarray(y_val, 'int32')),
#           validation_data=(x_val2, np.asarray(y_val[:, 1], 'int32')),
          shuffle=False, class_weight=class_weight,
          nb_epoch=50, batch_size=32, verbose=0, 
          callbacks=[checkpoint, callbacks])

model.save(FOLDER + '/my_model.h5')



print('--------------Start eva----------------'.format(datetime.datetime.now()))
model_name = sorted(glob.iglob(os.path.join(FOLDER, '*.h5')), key=os.path.getctime, reverse=True)[0]
model = load_model(model_name)
probas_ = model.predict(x_val2)
y_test = y_val[:, 1].astype(int)
max_accuracy, thresh = get_max_acc(y_test, probas_[:, 1])
pred = [1 if m > thresh else 0 for m in probas_[:, 1]]
# max_accuracy, thresh = get_max_acc(y_test, probas_)
# pred = [1 if m > thresh else 0 for m in probas_]
print('Fold {} accuracy = {:.2f}'.format(fold + 1, max_accuracy * 100))
print(classification_report(y_test, pred, target_names=['Control', 'Depress']))

df = pd.DataFrame(y_test, columns=['label'])
#         df['class0'] = probas_[:, 0]
df['class1'] = probas_[:, 1]
df.to_csv(MODEL_FOLDER + '/dl-' + str(INPUT_LENGTH) + '-' + str(EMBEDDING_DIM) + str(fold) + '.csv', index=False)

weighted = precision_recall_fscore_support(y_test, pred, average='weighted')
macro = precision_recall_fscore_support(y_test, pred, average='macro')
micro = precision_recall_fscore_support(y_test, pred, average='micro')
both = precision_recall_fscore_support(y_test, pred)
results.append(['dl-max-' + str(fold),
                max_accuracy, weighted[0], weighted[1], weighted[2],
                macro[0], macro[1], macro[2], micro[0], micro[1], micro[2],
                both[0][0], both[1][0], both[2][0], both[0][1], both[1][1], both[2][1]])

pred = [1 if m > 0.5 else 0 for m in probas_[:, 1]]
print('Fold {} normal accuracy = {:.2f}'.format(fold + 1, accuracy_score(y_test, pred) * 100))
max_accuracy = accuracy_score(y_test, pred)
print(classification_report(y_test, pred, target_names=['Control', 'Depress']))

weighted = precision_recall_fscore_support(y_test, pred, average='weighted')
macro = precision_recall_fscore_support(y_test, pred, average='macro')
micro = precision_recall_fscore_support(y_test, pred, average='micro')
both = precision_recall_fscore_support(y_test, pred)
results.append(['dl-nor-' + str(fold),
                max_accuracy, weighted[0], weighted[1], weighted[2],
                macro[0], macro[1], macro[2], micro[0], micro[1], micro[2],
                both[0][0], both[1][0], both[2][0], both[0][1], both[1][1], both[2][1]])

print('-------------------------------------------Start Best Model--------------------------------------------')

model_name = sorted(glob.iglob(os.path.join(FOLDER, '*.hdf5')), key=os.path.getctime, reverse=True)[0]
model = load_model(model_name)
probas_ = model.predict(x_val2)
y_test = y_val[:, 1].astype(int)
max_accuracy, thresh = get_max_acc(y_test, probas_[:, 1])
pred = [1 if m > thresh else 0 for m in probas_[:, 1]]
# max_accuracy, thresh = get_max_acc(y_test, probas_)
# pred = [1 if m > thresh else 0 for m in probas_]
print('Fold {} accuracy = {:.2f}'.format(fold + 1, max_accuracy * 100))
print(classification_report(y_test, pred, target_names=['Control', 'Depress']))

df = pd.DataFrame(y_test, columns=['label'])
#         df['class0'] = probas_[:, 0]
df['class1'] = probas_[:, 1]
df.to_csv(MODEL_FOLDER + '/dl-best-' + str(INPUT_LENGTH) + '-' + str(EMBEDDING_DIM) + str(fold) + '.csv', index=False)

weighted = precision_recall_fscore_support(y_test, pred, average='weighted')
macro = precision_recall_fscore_support(y_test, pred, average='macro')
micro = precision_recall_fscore_support(y_test, pred, average='micro')
both = precision_recall_fscore_support(y_test, pred)
results.append(['dl-best-max-' + str(fold),
                max_accuracy, weighted[0], weighted[1], weighted[2],
                macro[0], macro[1], macro[2], micro[0], micro[1], micro[2],
                both[0][0], both[1][0], both[2][0], both[0][1], both[1][1], both[2][1]])

pred = [1 if m > 0.5 else 0 for m in probas_[:, 1]]
print('Fold {} normal accuracy = {:.2f}'.format(fold + 1, accuracy_score(y_test, pred) * 100))
max_accuracy = accuracy_score(y_test, pred)
print(classification_report(y_test, pred, target_names=['Control', 'Depress']))

weighted = precision_recall_fscore_support(y_test, pred, average='weighted')
macro = precision_recall_fscore_support(y_test, pred, average='macro')
micro = precision_recall_fscore_support(y_test, pred, average='micro')
both = precision_recall_fscore_support(y_test, pred)
results.append(['dl-best-nor-' + str(fold),
                max_accuracy, weighted[0], weighted[1], weighted[2],
                macro[0], macro[1], macro[2], micro[0], micro[1], micro[2],
                both[0][0], both[1][0], both[2][0], both[0][1], both[1][1], both[2][1]])
K.clear_session()

model fitting - Hierachical LSTM
{0: 1.0, 1: 3.4231738035264483}
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 15000, 100)        5000000   
_________________________________________________________________
dropout_3 (Dropout)          (None, 15000, 100)        0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 14998, 250)        75250     
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 250)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 250)               62750     
_________________________________________________________________
activation_3 (Activation)    (None, 250)               0         
_________________________________________________________________
dropout_4 (

C:\Users\Bright\.conda\envs\py_37\lib\site-packages\ipykernel_launcher.py:90: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


--------------Start eva----------------
Fold 1 accuracy = 77.45
              precision    recall  f1-score   support

     Control       0.77      1.00      0.87       340
     Depress       0.00      0.00      0.00        99

    accuracy                           0.77       439
   macro avg       0.39      0.50      0.44       439
weighted avg       0.60      0.77      0.68       439

Fold 1 normal accuracy = 77.45
              precision    recall  f1-score   support

     Control       0.77      1.00      0.87       340
     Depress       0.00      0.00      0.00        99

    accuracy                           0.77       439
   macro avg       0.39      0.50      0.44       439
weighted avg       0.60      0.77      0.68       439

-------------------------------------------Start Best Model--------------------------------------------


C:\Users\Bright\.conda\envs\py_37\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fold 1 accuracy = 77.45
              precision    recall  f1-score   support

     Control       0.77      1.00      0.87       340
     Depress       0.00      0.00      0.00        99

    accuracy                           0.77       439
   macro avg       0.39      0.50      0.44       439
weighted avg       0.60      0.77      0.68       439

Fold 1 normal accuracy = 77.45
              precision    recall  f1-score   support

     Control       0.77      1.00      0.87       340
     Depress       0.00      0.00      0.00        99

    accuracy                           0.77       439
   macro avg       0.39      0.50      0.44       439
weighted avg       0.60      0.77      0.68       439



C:\Users\Bright\.conda\envs\py_37\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
def create_dl_model():
    num_filters = 250
    drop = 0.3

    embedding_layer = Embedding(num_words,
                                EMBEDDING_DIM,
                                input_length=10000,
#                                 weights=[embedding_matrix],
#                                 trainable=False)
                                trainable=True)

    model_input = Input(shape=(10000,), dtype='int32')
    model_layer = embedding_layer(model_input)
    model_layer = Dropout(drop)(model_layer)
    model_layer = Conv1D(filters=num_filters, kernel_size=3)(model_layer)
    model_layer = GlobalMaxPooling1D()(model_layer)
    # model_layer = Dropout(drop)(model_layer)
    model_layer = Dense(num_filters, activation='relu')(model_layer)
    model_layer = Dropout(drop)(model_layer)
#     model_layer = Dense(2, activation='softmax')(model_layer)
    model_layer = Dense(1, activation='sigmoid')(model_layer)
    model = Model(model_input, model_layer)

    print(model.summary())
    adam = Adam(lr=0.005, clipnorm=7.0)
    # adam = AdaMod()
    model.compile(loss='binary_crossentropy',
                  optimizer=adam,
                  metrics=['acc'])  # metrics.binary_accuracy
    return model



class_weight = get_class_weights(np.asarray(y_train, 'int32')[:, 1])

FOLDER = "{}/{}".format(MODEL_FOLDER, datetime.datetime.now().strftime("%Y-%m-%d %H-%M"))
if not os.path.exists(FOLDER):
    os.makedirs(FOLDER)
callbacks = TensorBoard(log_dir=FOLDER,
                        write_graph=True, write_grads=False, histogram_freq=0,
                        write_images=True, embeddings_freq=0, embeddings_layer_names='embedding_1',
                        embeddings_metadata=None)

checkpoint = ModelCheckpoint(FOLDER + '/model.{epoch:02d}-{val_acc:.2f}.hdf5',
                             verbose=0, monitor='val_acc',
                             save_best_only=True, mode='auto')

es = EarlyStopping(monitor='val_loss', mode='auto', verbose=0, patience=10)

print("model fitting - Hierachical LSTM")

print(class_weight)
model = create_dl_model()

X_train2 = np.zeros((len(X_train), 10000), dtype='int32')
for i, posts in enumerate(X_train):
    seq_data = pad_sequences([posts], maxlen=10000, padding='pre', truncating='post')
    X_train2[i] = seq_data
    
x_val2 = np.zeros((len(x_val), 10000), dtype='int32')
for i, posts in enumerate(x_val):
    seq_data = pad_sequences([posts], maxlen=10000, padding='pre', truncating='post')
    x_val2[i] = seq_data

# model.fit(X_train, np.asarray(y_train, 'int32'),
model.fit(X_train2, np.asarray(y_train[:, 1], 'int32'),
#           validation_data=(x_val, np.asarray(y_val, 'int32')),
          validation_data=(x_val2, np.asarray(y_val[:, 1], 'int32')),
          shuffle=False, 
          nb_epoch=200, batch_size=32, verbose=0, 
          callbacks=[checkpoint, callbacks, es])

model.save(FOLDER + '/my_model.h5')



print('--------------Start eva----------------'.format(datetime.datetime.now()))
model_name = sorted(glob.iglob(os.path.join(FOLDER, '*.h5')), key=os.path.getctime, reverse=True)[0]
model = load_model(model_name,
                   custom_objects={'optimizer': AdaMod})
probas_ = model.predict(x_val2)
y_test = y_val[:, 1].astype(int)
#         max_accuracy, thresh = get_max_acc(y_test, probas_[:, 1])
#         pred = [1 if m > thresh else 0 for m in probas_[:, 1]]
max_accuracy, thresh = get_max_acc(y_test, probas_)
pred = [1 if m > thresh else 0 for m in probas_]
print('Fold {} accuracy = {:.2f}'.format(fold + 1, max_accuracy * 100))
print(classification_report(y_test, pred, target_names=['Control', 'Depress']))

df = pd.DataFrame(y_test, columns=['label'])
#         df['class0'] = probas_[:, 0]
df['class1'] = probas_
df.to_csv(MODEL_FOLDER + '/dl-' + str(INPUT_LENGTH) + '-' + str(EMBEDDING_DIM) + str(fold) + '.csv', index=False)

weighted = precision_recall_fscore_support(y_test, pred, average='weighted')
macro = precision_recall_fscore_support(y_test, pred, average='macro')
micro = precision_recall_fscore_support(y_test, pred, average='micro')
both = precision_recall_fscore_support(y_test, pred)
results.append(['dl-max-' + str(fold),
                max_accuracy, weighted[0], weighted[1], weighted[2],
                macro[0], macro[1], macro[2], micro[0], micro[1], micro[2],
                both[0][0], both[1][0], both[2][0], both[0][1], both[1][1], both[2][1]])

pred = [1 if m > 0.5 else 0 for m in probas_]
print('Fold {} normal accuracy = {:.2f}'.format(fold + 1, accuracy_score(y_test, pred) * 100))
max_accuracy = accuracy_score(y_test, pred)
print(classification_report(y_test, pred, target_names=['Control', 'Depress']))

weighted = precision_recall_fscore_support(y_test, pred, average='weighted')
macro = precision_recall_fscore_support(y_test, pred, average='macro')
micro = precision_recall_fscore_support(y_test, pred, average='micro')
both = precision_recall_fscore_support(y_test, pred)
results.append(['dl-nor-' + str(fold),
                max_accuracy, weighted[0], weighted[1], weighted[2],
                macro[0], macro[1], macro[2], micro[0], micro[1], micro[2],
                both[0][0], both[1][0], both[2][0], both[0][1], both[1][1], both[2][1]])

print('-------------------------------------------Start Best Model--------------------------------------------')

model_name = sorted(glob.iglob(os.path.join(FOLDER, '*.hdf5')), key=os.path.getctime, reverse=True)[0]
model = load_model(model_name,
                   custom_objects={'optimizer': AdaMod})
probas_ = model.predict(x_val2)
y_test = y_val[:, 1].astype(int)
#         max_accuracy, thresh = get_max_acc(y_test, probas_[:, 1])
#         pred = [1 if m > thresh else 0 for m in probas_[:, 1]]
max_accuracy, thresh = get_max_acc(y_test, probas_)
pred = [1 if m > thresh else 0 for m in probas_]
print('Fold {} accuracy = {:.2f}'.format(fold + 1, max_accuracy * 100))
print(classification_report(y_test, pred, target_names=['Control', 'Depress']))

df = pd.DataFrame(y_test, columns=['label'])
#         df['class0'] = probas_[:, 0]
df['class1'] = probas_
df.to_csv(MODEL_FOLDER + '/dl-best-' + str(INPUT_LENGTH) + '-' + str(EMBEDDING_DIM) + str(fold) + '.csv', index=False)

weighted = precision_recall_fscore_support(y_test, pred, average='weighted')
macro = precision_recall_fscore_support(y_test, pred, average='macro')
micro = precision_recall_fscore_support(y_test, pred, average='micro')
both = precision_recall_fscore_support(y_test, pred)
results.append(['dl-best-max-' + str(fold),
                max_accuracy, weighted[0], weighted[1], weighted[2],
                macro[0], macro[1], macro[2], micro[0], micro[1], micro[2],
                both[0][0], both[1][0], both[2][0], both[0][1], both[1][1], both[2][1]])

pred = [1 if m > 0.5 else 0 for m in probas_]
print('Fold {} normal accuracy = {:.2f}'.format(fold + 1, accuracy_score(y_test, pred) * 100))
max_accuracy = accuracy_score(y_test, pred)
print(classification_report(y_test, pred, target_names=['Control', 'Depress']))

weighted = precision_recall_fscore_support(y_test, pred, average='weighted')
macro = precision_recall_fscore_support(y_test, pred, average='macro')
micro = precision_recall_fscore_support(y_test, pred, average='micro')
both = precision_recall_fscore_support(y_test, pred)
results.append(['dl-best-nor-' + str(fold),
                max_accuracy, weighted[0], weighted[1], weighted[2],
                macro[0], macro[1], macro[2], micro[0], micro[1], micro[2],
                both[0][0], both[1][0], both[2][0], both[0][1], both[1][1], both[2][1]])


model fitting - Hierachical LSTM
{0: 1.0, 1: 3.4231738035264483}
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_25 (InputLayer)        (None, 10000)             0         
_________________________________________________________________
embedding_25 (Embedding)     (None, 10000, 100)        5000000   
_________________________________________________________________
dropout_47 (Dropout)         (None, 10000, 100)        0         
_________________________________________________________________
conv1d_24 (Conv1D)           (None, 9998, 250)         75250     
_________________________________________________________________
global_max_pooling1d_24 (Glo (None, 250)               0         
_________________________________________________________________
dense_47 (Dense)             (None, 250)               62750     
_________________________________________________________________
dropout_48 

C:\Users\Bright\.conda\envs\py_37\lib\site-packages\ipykernel_launcher.py:71: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


--------------Start eva----------------
Fold 1 accuracy = 85.88
              precision    recall  f1-score   support

     Control       0.86      0.97      0.91       340
     Depress       0.82      0.47      0.60        99

    accuracy                           0.86       439
   macro avg       0.84      0.72      0.76       439
weighted avg       0.86      0.86      0.84       439

Fold 1 normal accuracy = 83.14
              precision    recall  f1-score   support

     Control       0.90      0.89      0.89       340
     Depress       0.62      0.65      0.63        99

    accuracy                           0.83       439
   macro avg       0.76      0.77      0.76       439
weighted avg       0.83      0.83      0.83       439

-------------------------------------------Start Best Model--------------------------------------------
Fold 1 accuracy = 86.79
              precision    recall  f1-score   support

     Control       0.88      0.96      0.92       340
     Depress  